<a href="https://colab.research.google.com/github/HarryWightJr/final_project/blob/main/data_cleaning_s3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:15 http://ppa

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-24 21:39:52--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.05MB/s    in 0.2s    

2022-03-24 21:39:53 (6.05 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url_1 = "https://bootcampfinalproject.s3.amazonaws.com/Resources/Crime_2019.csv"
url_2 = "https://bootcampfinalproject.s3.amazonaws.com/Resources/Crime_2020.csv"
url_3 = "https://bootcampfinalproject.s3.amazonaws.com/Resources/Sale_Prices_City.csv"
url_4 = "https://bootcampfinalproject.s3.amazonaws.com/Resources/Income.csv"
spark.sparkContext.addFile(url_1)
crime_2019_df = spark.read.csv(SparkFiles.get("Crime_2019.csv"), sep=",", header=True)
spark.sparkContext.addFile(url_2)
crime_2020_df = spark.read.csv(SparkFiles.get("Crime_2020.csv"), sep=",", header=True)
spark.sparkContext.addFile(url_3)
city_sales_df = spark.read.csv(SparkFiles.get("Sale_Prices_City.csv"), sep=",", header=True)
spark.sparkContext.addFile(url_4)
income_df = spark.read.csv(SparkFiles.get("Income.csv"), sep=",", header=True)

In [5]:
crime_2019_df.show()

+----------+-------+--------------+--------+---------------+------------------------+-------------------------+
|      City|  State|City and State|CountyID|Population 2019|Violent Crime Total 2019|Property Crime Total 2019|
+----------+-------+--------------+--------+---------------+------------------------+-------------------------+
|    Hoover|ALABAMA|    Hoover, AL|    1073|          85670|                     114|                     1922|
| Anchorage| ALASKA| Anchorage, AK|    2020|         287731|                    3581|                    12261|
|    Bethel| ALASKA|    Bethel, AK|    2050|           6544|                     130|                      132|
|   Cordova| ALASKA|   Cordova, AK|    2261|           2150|                       0|                        7|
|     Craig| ALASKA|     Craig, AK|    2198|           1313|                       7|                       20|
|Dillingham| ALASKA|Dillingham, AK|    2070|           2405|                      49|                   

In [6]:
dropped_crime_2019_df = crime_2019_df.drop("City", "State","City and State")
sorted_crime_2019_df = dropped_crime_2019_df.sort("CountyID")
final_crime_2019_df = sorted_crime_2019_df.dropDuplicates(["CountyID"])
final_crime_2019_df.show()

+--------+---------------+------------------------+-------------------------+
|CountyID|Population 2019|Violent Crime Total 2019|Property Crime Total 2019|
+--------+---------------+------------------------+-------------------------+
|   10001|           3507|                       5|                      173|
|   10003|           1830|                      11|                       52|
|   10005|           1244|                       3|                       79|
|    1073|          85670|                     114|                     1922|
|   11001|         705749|                    6896|                    29965|
|   12001|          10070|                      41|                      230|
|   12005|          21859|                      29|                      310|
|   12007|            722|                       4|                        5|
|   12009|          18807|                     151|                      965|
|   12011|          62471|                      82|             

In [7]:
final_crime_2019_df.count()

2153

In [8]:
crime_2020_df.show()

+------------+-------+----------------+--------+---------------+------------------------+-------------------------+
|        City|  State|  City and State|CountyID|Population 2020|Violent Crime Total 2020|Property Crime Total 2020|
+------------+-------+----------------+--------+---------------+------------------------+-------------------------+
| Cedar Bluff|Alabama| Cedar Bluff, AL|    1019|           1823|                       4|                       36|
|      Centre|Alabama|      Centre, AL|    1019|           3547|                      20|                      124|
|   Daleville|Alabama|   Daleville, AL|    1045|           5080|                      16|                       98|
|  Enterprise|Alabama|  Enterprise, AL|    1031|          28569|                     128|                      715|
|     Eufaula|Alabama|     Eufaula, AL|    1005|          11568|                      95|                      456|
|     Glencoe|Alabama|     Glencoe, AL|    1055|           5063|        

In [9]:
dropped_crime_2020_df = crime_2020_df.drop("City", "State","City and State")
sorted_crime_2020_df = dropped_crime_2020_df.sort("CountyID")
final_crime_2020_df = sorted_crime_2020_df.dropDuplicates(["CountyID"])
final_crime_2020_df.show()

+--------+---------------+------------------------+-------------------------+
|CountyID|Population 2020|Violent Crime Total 2020|Property Crime Total 2020|
+--------+---------------+------------------------+-------------------------+
|   10001|           3562|                      11|                      209|
|   10003|           1844|                      12|                       31|
|   10005|           1268|                       0|                       63|
|    1005|          11568|                      95|                      456|
|    1015|           4501|                      23|                       92|
|    1019|           1823|                       4|                       36|
|    1025|           4583|                      29|                       83|
|    1031|          28569|                     128|                      715|
|    1033|            754|                      12|                       36|
|    1045|           5080|                      16|             

In [10]:
final_crime_2020_df.count()

2113

In [11]:
city_sales_df.show()

+--------------------+--------------+--------+---------------------+---------------------+
|          RegionName|     StateName|CountyID|AverageSalesPrice2019|AverageSalesPrice2020|
+--------------------+--------------+--------+---------------------+---------------------+
|      Youngstown, OH|          Ohio|   39099|                29625|                31300|
|          Harvey, IL|      Illinois|   17031|                35183|                36000|
|         Trenton, NJ|    New Jersey|   34021|                51833|                56450|
|   Maple Heights, OH|          Ohio|   39035|                58692|                70567|
|        Bay City, MI|      Michigan|   26017|                59150|                64300|
|Garfield Heights, OH|          Ohio|   39035|                62817|                67467|
|       Cleveland, OH|          Ohio|   39035|                62925|                67350|
|    Granite City, IL|      Illinois|   17119|                65667|                65400|

In [12]:
city_sales_df.withColumn("CountyID",city_sales_df.CountyID.cast('int'))

DataFrame[RegionName: string, StateName: string, CountyID: int, AverageSalesPrice2019: string, AverageSalesPrice2020: string]

In [13]:
city_sales_df.show()

+--------------------+--------------+--------+---------------------+---------------------+
|          RegionName|     StateName|CountyID|AverageSalesPrice2019|AverageSalesPrice2020|
+--------------------+--------------+--------+---------------------+---------------------+
|      Youngstown, OH|          Ohio|   39099|                29625|                31300|
|          Harvey, IL|      Illinois|   17031|                35183|                36000|
|         Trenton, NJ|    New Jersey|   34021|                51833|                56450|
|   Maple Heights, OH|          Ohio|   39035|                58692|                70567|
|        Bay City, MI|      Michigan|   26017|                59150|                64300|
|Garfield Heights, OH|          Ohio|   39035|                62817|                67467|
|       Cleveland, OH|          Ohio|   39035|                62925|                67350|
|    Granite City, IL|      Illinois|   17119|                65667|                65400|

In [14]:
dropped_city_sales_df = city_sales_df.drop("RegionName", "StateName")
final_city_sales_df = dropped_city_sales_df.dropDuplicates(["CountyID"])
final_city_sales_df.show()

+--------+---------------------+---------------------+
|CountyID|AverageSalesPrice2019|AverageSalesPrice2020|
+--------+---------------------+---------------------+
|   10001|               180117|               189500|
|   10003|               177100|               187950|
|   10005|               151633|               167100|
|    1003|               172500|               173833|
|    1015|               124467|               132733|
|    1021|               137167|               149933|
|    1031|               146258|               162033|
|    1043|               155033|               148633|
|    1051|               131142|               149367|
|    1069|               146817|               148333|
|    1073|                94425|               100833|
|    1077|               118333|               118067|
|    1083|               165883|               172033|
|    1089|               158750|               199533|
|    1095|               110550|               139900|
|    1097|

In [15]:
final_city_sales_df.count()

994

In [16]:
income_df.show()

+---------+-------+------------------+--------+-----------+-----------+
|   County|  State|     County, State|CountyID|Income_2019|Income_2020|
+---------+-------+------------------+--------+-----------+-----------+
|  Autauga|Alabama|  Autauga, Alabama|    1001|      44368|      46814|
|  Baldwin|Alabama|  Baldwin, Alabama|    1003|      48270|      50953|
|  Barbour|Alabama|  Barbour, Alabama|    1005|      35572|      37850|
|     Bibb|Alabama|     Bibb, Alabama|    1007|      31540|      34300|
|   Blount|Alabama|   Blount, Alabama|    1009|      36969|      38808|
|  Bullock|Alabama|  Bullock, Alabama|    1011|      28136|      31944|
|   Butler|Alabama|   Butler, Alabama|    1013|      38073|      39988|
|  Calhoun|Alabama|  Calhoun, Alabama|    1015|      37708|      40195|
| Chambers|Alabama| Chambers, Alabama|    1017|      36110|      38508|
| Cherokee|Alabama| Cherokee, Alabama|    1019|      36381|      37869|
|  Chilton|Alabama|  Chilton, Alabama|    1021|      36471|     

In [17]:
dropped_income_df = income_df.drop("County", "State","County, State")
final_income_df = dropped_income_df.dropDuplicates(["CountyID"])
final_income_df.show()

+--------+-----------+-----------+
|CountyID|Income_2019|Income_2020|
+--------+-----------+-----------+
|   10001|      43761|      46600|
|   10003|      57404|      59955|
|   10005|      52935|      54352|
|    1001|      44368|      46814|
|    1003|      48270|      50953|
|    1005|      35572|      37850|
|    1007|      31540|      34300|
|    1009|      36969|      38808|
|    1011|      28136|      31944|
|    1013|      38073|      39988|
|    1015|      37708|      40195|
|    1017|      36110|      38508|
|    1019|      36381|      37869|
|    1021|      36471|      38778|
|    1023|      39097|      42231|
|    1025|      37061|      40822|
|    1027|      34185|      37179|
|    1029|      35905|      37734|
|    1031|      43319|      45262|
|    1033|      38860|      41941|
+--------+-----------+-----------+
only showing top 20 rows



In [18]:
final_income_df.count()

3038

In [19]:
# Combine the data into a single dataset.
partial_1_df = final_city_sales_df.join(final_crime_2019_df, on="CountyID", how="left")
partial_1_df.show()

+--------+---------------------+---------------------+---------------+------------------------+-------------------------+
|CountyID|AverageSalesPrice2019|AverageSalesPrice2020|Population 2019|Violent Crime Total 2019|Property Crime Total 2019|
+--------+---------------------+---------------------+---------------+------------------------+-------------------------+
|   10001|               180117|               189500|           3507|                       5|                      173|
|   10003|               177100|               187950|           1830|                      11|                       52|
|   10005|               151633|               167100|           1244|                       3|                       79|
|    1003|               172500|               173833|           null|                    null|                     null|
|    1015|               124467|               132733|           null|                    null|                     null|
|    1021|              

In [20]:
partial_2_df = partial_1_df.join(final_crime_2020_df, how="left", on="CountyID")
partial_2_df.show()

+--------+---------------------+---------------------+---------------+------------------------+-------------------------+---------------+------------------------+-------------------------+
|CountyID|AverageSalesPrice2019|AverageSalesPrice2020|Population 2019|Violent Crime Total 2019|Property Crime Total 2019|Population 2020|Violent Crime Total 2020|Property Crime Total 2020|
+--------+---------------------+---------------------+---------------+------------------------+-------------------------+---------------+------------------------+-------------------------+
|   10001|               180117|               189500|           3507|                       5|                      173|           3562|                      11|                      209|
|   10003|               177100|               187950|           1830|                      11|                       52|           1844|                      12|                       31|
|   10005|               151633|               167100| 

In [21]:
complete_df = partial_2_df.join(final_income_df, how="left", on="CountyID")
complete_df.show()

+--------+---------------------+---------------------+---------------+------------------------+-------------------------+---------------+------------------------+-------------------------+-----------+-----------+
|CountyID|AverageSalesPrice2019|AverageSalesPrice2020|Population 2019|Violent Crime Total 2019|Property Crime Total 2019|Population 2020|Violent Crime Total 2020|Property Crime Total 2020|Income_2019|Income_2020|
+--------+---------------------+---------------------+---------------+------------------------+-------------------------+---------------+------------------------+-------------------------+-----------+-----------+
|   10001|               180117|               189500|           3507|                       5|                      173|           3562|                      11|                      209|      43761|      46600|
|   10003|               177100|               187950|           1830|                      11|                       52|           1844|           

In [22]:
complete_df.count()

994

In [23]:
# Print our schema
complete_df.printSchema()

root
 |-- CountyID: string (nullable = true)
 |-- AverageSalesPrice2019: string (nullable = true)
 |-- AverageSalesPrice2020: string (nullable = true)
 |-- Population 2019: string (nullable = true)
 |-- Violent Crime Total 2019: string (nullable = true)
 |-- Property Crime Total 2019: string (nullable = true)
 |-- Population 2020: string (nullable = true)
 |-- Violent Crime Total 2020: string (nullable = true)
 |-- Property Crime Total 2020: string (nullable = true)
 |-- Income_2019: string (nullable = true)
 |-- Income_2020: string (nullable = true)



In [24]:
complete_df = complete_df.dropna(subset = ['Population 2019', 'Violent Crime Total 2019', 'Property Crime Total 2019','Population 2020', 'Violent Crime Total 2020', 'Property Crime Total 2020', 'Income_2019', 'Income_2020'])

In [25]:
complete_df.show()

+--------+---------------------+---------------------+---------------+------------------------+-------------------------+---------------+------------------------+-------------------------+-----------+-----------+
|CountyID|AverageSalesPrice2019|AverageSalesPrice2020|Population 2019|Violent Crime Total 2019|Property Crime Total 2019|Population 2020|Violent Crime Total 2020|Property Crime Total 2020|Income_2019|Income_2020|
+--------+---------------------+---------------------+---------------+------------------------+-------------------------+---------------+------------------------+-------------------------+-----------+-----------+
|   10001|               180117|               189500|           3507|                       5|                      173|           3562|                      11|                      209|      43761|      46600|
|   10003|               177100|               187950|           1830|                      11|                       52|           1844|           

In [26]:
complete_df.count()

764

In [27]:
complete_df.printSchema()

root
 |-- CountyID: string (nullable = true)
 |-- AverageSalesPrice2019: string (nullable = true)
 |-- AverageSalesPrice2020: string (nullable = true)
 |-- Population 2019: string (nullable = true)
 |-- Violent Crime Total 2019: string (nullable = true)
 |-- Property Crime Total 2019: string (nullable = true)
 |-- Population 2020: string (nullable = true)
 |-- Violent Crime Total 2020: string (nullable = true)
 |-- Property Crime Total 2020: string (nullable = true)
 |-- Income_2019: string (nullable = true)
 |-- Income_2020: string (nullable = true)



In [28]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [31]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cx6o5jgyfq1i.us-east-1.rds.amazonaws.com:5432/database-1"
config = {"user":"postgres",
          "password": "bootcamp",
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [32]:
# Write DataFrame to active_user table in RDS
complete_df.write.jdbc(url=jdbc_url, table='complete_df', mode=mode, properties=config)

Py4JJavaError: ignored